In [19]:
import json

with open('countries.json', 'r') as file:
    countryList = json.load(file)["countries"]


url_template = "https://gadebate.un.org/sites/default/files/gastatements/79/{code}_en.pdf"
pdf_urls = [url_template.format(code=country['iso_code'].lower()) for country in countryList]

print(pdf_urls)

['https://gadebate.un.org/sites/default/files/gastatements/79/ad_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/ae_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/af_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/ag_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/al_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/am_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/ao_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/ar_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/at_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/au_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/az_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/ba_en.pdf', 'https://gadebate.un.org/sites/default/files/gastatements/79/bb_en.pdf', 'https://gadebate.un.org/sites/default/files/gasta

In [20]:
%pip install requests -q

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import requests
import time

os.makedirs('./speeches', exist_ok=True)

for url in pdf_urls[:10]:
    try:
        response = requests.get(url)
        response.raise_for_status()
        filename = os.path.join('./speeches', url.split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
        time.sleep(0.2)
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {url}: {e}")


Failed to download https://gadebate.un.org/sites/default/files/gastatements/79/ad_en.pdf: 404 Client Error: Not Found for url: https://gadebate.un.org/sites/default/files/gastatements/79/ad_en.pdf
Downloaded: ./speeches/ae_en.pdf
Failed to download https://gadebate.un.org/sites/default/files/gastatements/79/af_en.pdf: 404 Client Error: Not Found for url: https://gadebate.un.org/sites/default/files/gastatements/79/af_en.pdf
Downloaded: ./speeches/ag_en.pdf
Downloaded: ./speeches/al_en.pdf
Downloaded: ./speeches/am_en.pdf
Downloaded: ./speeches/ao_en.pdf
Failed to download https://gadebate.un.org/sites/default/files/gastatements/79/ar_en.pdf: 404 Client Error: Not Found for url: https://gadebate.un.org/sites/default/files/gastatements/79/ar_en.pdf
Downloaded: ./speeches/at_en.pdf
Downloaded: ./speeches/au_en.pdf


In [22]:
%pip install pytesseract pillow pdf2image -q

Note: you may need to restart the kernel to use updated packages.


In [23]:
import pytesseract
from pdf2image import convert_from_path
import glob

# Get list of PDF files in the folder
pdf_files = glob.glob('./speeches/*.pdf')

for pdf_file in pdf_files:
    # Convert PDF to images
    images = convert_from_path(pdf_file)
    
    # Perform OCR on each image and collect the text
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    
    # Write the text to a file with the same name but .txt extension
    txt_file = pdf_file.replace('.pdf', '.txt')
    with open(txt_file, 'w') as f:
        f.write(text)
    
    print(f"Converted {pdf_file} to {txt_file}")

Converted ./speeches/al_en.pdf to ./speeches/al_en.txt
Converted ./speeches/at_en.pdf to ./speeches/at_en.txt
Converted ./speeches/ao_en.pdf to ./speeches/ao_en.txt
Converted ./speeches/au_en.pdf to ./speeches/au_en.txt
Converted ./speeches/am_en.pdf to ./speeches/am_en.txt
Converted ./speeches/ae_en.pdf to ./speeches/ae_en.txt
Converted ./speeches/ag_en.pdf to ./speeches/ag_en.txt
